# Check-worthiness detection using Large Language Models

First, the necessary python modules are imported

In [1]:
%load_ext autoreload

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from claimbuster_utils import load_claimbuster_dataset
from checkthat_utils import load_check_that_dataset
from tqdm.auto import tqdm
import json
import numpy as np
import re
import torch
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.base import BaseEstimator, TransformerMixin

/cluster/home/matssbra/.conda/envs/fakeNews/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-03 18:48:48.343057: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-03 18:48:52.102937: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 18:48:52.121608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03

## Load model

In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype=torch.float16,
    quantization_config = bnb_config,
    # attn_implementation="flash_attention_2", 
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    return_full_text=False,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)


Loading checkpoint shards: 100%|██████████| 3/3 [00:24<00:00,  8.03s/it]


## Zero-shot classification

### ClaimBuster

In [4]:
with open("../prompts/ClaimBuster/standard/zero-shot.txt", "r") as f:
    instruction = f.read().replace("\n", "")
use_contextual = False
data = load_claimbuster_dataset(
    "../data/ClaimBuster_Datasets/datasets",
    use_contextual_features=use_contextual,
    debate_transcripts_folder="../data/ClaimBuster_Datasets/debate_transcripts",
)

texts = data["Text"]
if use_contextual is False:
    prompts = [f"{instruction} '''{text}'''" for text in texts]
    zeroshot_output = "../results/ClaimBuster/zeroshot1.csv"
else:
    contexts = data["previous_sentences"].tolist()
    prompts = [
        f"{instruction} For context, the following senteces were said prior to the one in question: {context} Only evaluate the check-worthiness of the following sentence: '''{text}'''"
        for text, context in zip(texts, contexts)
    ]
    zeroshot_output = "../results/ClaimBuster/zeroshot_contextual.csv"


class ProgressDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, idx):
        return self.dataset[idx]

    def __len__(self):
        return len(self.dataset)


prompts_data = ProgressDataset(prompts)

dataset_with_scores = data.copy()

display(data.head())
dict_matcher = re.compile(r"{.*}")
score_matcher = re.compile(r"([Ss]core[^\d]*)(\d+)")
non_check_worthy_matcher = re.compile(
    r"(non-checkworthy)|(not check-worthy)|(non check-worthy)"
)

responses = pipe(prompts_data, batch_size=128)
for index, result in enumerate(tqdm(responses, total=len(prompts))):
    response = result[0]["generated_text"].replace("\n", "")
    dataset_index = data.index[index]
    try:
        parsed_json = json.loads(dict_matcher.search(response).group(0))
        dataset_with_scores.loc[dataset_index, "score"] = parsed_json["score"]
        dataset_with_scores.loc[dataset_index, "reasoning"] = parsed_json["reasoning"]
    except (json.decoder.JSONDecodeError, AttributeError) as e:
        # Try to find score
        score = score_matcher.search(response)
        if score is not None:
            score = score[2]
        else:
            score = 0.0 if non_check_worthy_matcher.search(response) else np.nan
        dataset_with_scores.loc[dataset_index, "score"] = score
        dataset_with_scores.loc[dataset_index, "reasoning"] = response
        continue
# Set the following column order: Verdict, score, Text, reasoning, previous_sentences
columns =  ["Verdict", "score", "Text", "reasoning"]
if use_contextual:
    columns.append("previous_sentences")
dataset_with_scores = dataset_with_scores[columns]
dataset_with_scores.to_csv(zeroshot_output, index=True)

,Verdict,Text
sentence_id,,
27247,1,We're 9 million jobs short of that.
10766,1,"You know, last year up to this time, we've los..."
3327,1,And in November of 1975 I was the first presid...
19700,1,And what we've done during the Bush administra...
12600,1,Do you know we don't have a single program spo...


  3%|▎         | 298/9674 [01:42<39:52,  3.92it/s]  /tmp/ipykernel_4125055/2661599538.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataset_with_scores.loc[dataset_index, "score"] = score
100%|██████████| 9674/9674 [31:28<00:00,  5.12it/s]


#### Discussion of results

In [43]:
# Print the number of empty scores
dataset_path = "../results/ClaimBuster/zeroshot_contextual.csv"
dataset_with_scores = pd.read_csv(dataset_path, index_col=0)
class ThresholdOptimizer(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.threshold = None

    def fit(self, x: pd.DataFrame, y: pd.Series):
        
        y_gold = x["Verdict"].values

        reports = []
        for threshold in range(1, 100):
            y_pred = x["score"].map(lambda x: 1 if x >= threshold else 0).values
            report = classification_report(y_gold, y_pred, output_dict=True)
            report["threshold"] = threshold
            reports.append(report)
        self.threshold = max(reports, key=lambda report: report["macro avg"]["f1-score"])["threshold"]
    
    def predict(self, x: pd.DataFrame):
        predictions = x["score"].map(lambda x: 1 if x >= self.threshold else 0)
        return predictions

# Do a four fold cross validation where the threshold is optimized
print(cross_validate(
    ThresholdOptimizer(),
    X=dataset_with_scores,
    y=dataset_with_scores["Verdict"],
    cv=StratifiedKFold(n_splits=4),
    scoring=["f1_macro", "accuracy"],
))

{'fit_time': array([1.28376102, 1.50472569, 1.22792745, 1.2808435 ]), 'score_time': array([0.00417757, 0.0038116 , 0.00471807, 0.00366807]), 'test_f1_macro': array([0.64137709, 0.63468915, 0.62384077, 0.62942657]), 'test_accuracy': array([0.68168665, 0.68582059, 0.66956162, 0.67162945])}


### CheckThat 2021 Task 1a Tweets

In [3]:
%autoreload
with open("../prompts/ClaimBuster/standard/zero-shot.txt", "r") as f:
    instruction = f.read().replace("\n", "")
use_contextual = False
data = load_check_that_dataset(
    "../data/CheckThat2021Task1a",
)

texts = data["tweet_text"]
if use_contextual is False:
    prompts = [f"{instruction} '''{text}'''" for text in texts]
    zeroshot_output = "../results/CheckThat/zeroshot1.csv"
else:
    contexts = data["previous_sentences"].tolist()
    prompts = [
        f"{instruction} For context, the following senteces were said prior to the one in question: {context} Only evaluate the check-worthiness of the following sentence: '''{text}'''"
        for text, context in zip(texts, contexts)
    ]
    zeroshot_output = "../results/Checkthat/zeroshot_contextual.csv"


class ProgressDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, idx):
        return self.dataset[idx]

    def __len__(self):
        return len(self.dataset)


prompts_data = ProgressDataset(prompts)

dataset_with_scores = data.copy()

display(data.head())
dict_matcher = re.compile(r"{.*}")
score_matcher = re.compile(r"([Ss]core[^\d]*)(\d+)")
non_check_worthy_matcher = re.compile(
    r"(non-checkworthy)|(not check-worthy)|(non check-worthy)"
)

responses = pipe(prompts_data, batch_size=128)
for index, result in enumerate(tqdm(responses, total=len(prompts))):
    response = result[0]["generated_text"].replace("\n", "")
    dataset_index = data.index[index]
    try:
        parsed_json = json.loads(dict_matcher.search(response).group(0))
        dataset_with_scores.loc[dataset_index, "score"] = parsed_json["score"]
        dataset_with_scores.loc[dataset_index, "reasoning"] = parsed_json["reasoning"]
    except (json.decoder.JSONDecodeError, AttributeError) as e:
        # Try to find score
        score = score_matcher.search(response)
        if score is not None:
            score = score[2]
        else:
            score = 0.0 if non_check_worthy_matcher.search(response) else np.nan
        dataset_with_scores.loc[dataset_index, "score"] = score
        dataset_with_scores.loc[dataset_index, "reasoning"] = response
        continue
columns =  ["check_worthiness", "score", "tweet_text", "reasoning"]
if use_contextual:
    columns.append("previous_sentences")
dataset_with_scores = dataset_with_scores[columns]
dataset_with_scores.to_csv(zeroshot_output, index=True)

,topic_id,tweet_url,tweet_text,claim,check_worthiness
tweet_id,,,,,
1234964653014384644,covid-19,https://twitter.com/EricTrump/status/123496465...,Since this will never get reported by the medi...,1,1
1234869939720216578,covid-19,https://twitter.com/RealJamesWoods/status/1234...,"Thanks, #MichaelBloomberg. Here’s a handy litt...",0,0
1234873136304267267,covid-19,https://twitter.com/hayxsmith/status/123487313...,"Folks, when you say ""The corona virus isn't a ...",0,0
1235071285027147776,covid-19,https://twitter.com/ipspankajnain/status/12350...,Just 1 case of Corona Virus in India and peop...,1,0
1234911110861594624,covid-19,https://twitter.com/PressSec/status/1234911110...,President @realDonaldTrump made a commitment...,1,1


  0%|          | 1/1172 [00:46<15:14:45, 46.87s/it]/tmp/ipykernel_10891/1019148809.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataset_with_scores.loc[dataset_index, "score"] = score
100%|██████████| 1172/1172 [07:01<00:00,  2.78it/s]


#### Discussion of results

In [5]:
dataset_path = "../results/CheckThat/zeroshot1.csv"
dataset_with_scores = pd.read_csv(dataset_path, index_col=0)
class ThresholdOptimizer(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.threshold = None

    def fit(self, x: pd.DataFrame, y: pd.Series):
        
        y_gold = x["check_worthiness"].values

        reports = []
        for threshold in range(1, 100):
            y_pred = x["score"].map(lambda x: 1 if x >= threshold else 0).values
            report = classification_report(y_gold, y_pred, output_dict=True)
            report["threshold"] = threshold
            reports.append(report)
        self.threshold = max(reports, key=lambda report: report["macro avg"]["f1-score"])["threshold"]
    
    def predict(self, x: pd.DataFrame):
        predictions = x["score"].map(lambda x: 1 if x >= self.threshold else 0)
        return predictions

# Do a four fold cross validation where the threshold is optimized
print(cross_validate(
    ThresholdOptimizer(),
    X=dataset_with_scores,
    y=dataset_with_scores["check_worthiness"],
    cv=StratifiedKFold(n_splits=4),
    scoring=["f1_macro", "accuracy"],
))

{'fit_time': array([0.5817163 , 0.554106  , 0.56280541, 0.57334805]), 'score_time': array([0.00258136, 0.00253034, 0.00249505, 0.00250578]), 'test_f1_macro': array([0.64136111, 0.7040404 , 0.50021622, 0.33657796]), 'test_accuracy': array([0.69283276, 0.74061433, 0.51535836, 0.33788396])}
